In [2]:
import logging
from context import pyTTRPGsimulator as rpg
rpg.setup_logging(logging.INFO)


%reload_ext autoreload
%autoreload 2

This is a test error message from module actors


In [3]:
attributes = rpg.actor_attributes

print(attributes)
print(attributes + attributes)
print(attributes == attributes)

Attributes(physical_defense=8, mystical_defense=8, physical_damage_reduction=0, mystical_damage_reduction=0, max_health_points=10, max_stamina_points=0, max_grit_points=0, max_mana_points=0, might=0, agility=0, intelligence=0, charisma=0, prime_modifier_bonus=0, might_save=0, agility_save=0, intelligence_save=0, charisma_save=0, might_check=0, agility_check=0, intelligence_check=0, charisma_check=0, initiative=0, move_speed=5, might_adv=0, agility_adv=0, intelligence_adv=0, charisma_adv=0, combat_mastery=0, spell_DC=0, max_action_points=4, max_attack_before_penalty=1, critical_hit_damage=2, critical_hit_threshold=20, heavy_hit_damage=1, brutal_hit_damage=1, death_threshold=0, mastery_light_armor=True, mastery_heavy_armor=False, is_magic=False, is_bleeding=False, is_blinded=False, is_burning=False, is_charmed=False, is_dazed=False, is_deafened=False, is_doomed=False, is_exposed=False, is_frightened=False, is_grappled=False, is_hindered=False, is_impaired=False, is_incapacitated=False, i

In [4]:
# Create a basic sword
sword_dmg = rpg.Damage(damage_type=rpg.Slashing(), 
                        value=2)
sword = rpg.Sword(name="Basic sword",
        damages=sword_dmg)

# Create some actors (PC and monster)
actor_1 = rpg.Actor(name="Coach", items=sword)
actor_2 = rpg.Actor(name="Penguin King", items=sword)

# Initialize the combat manager
combat_manager = rpg.CombatManager([actor_1], [actor_2], initiative_dc=10)

# Run the combat
combat_manager.run_combat()

# Print the outcome
print(combat_manager.fight_debrief())


Rolling for initiative:
Coach gets 6
Penguin King gets 17

Turn order determined:
1/ Penguin King
2/ Coach


################ NEW ROUND ################
Alive actors at round 0:
    Coach, HPs: 10/10
    Penguin King, HPs: 10/10

Penguin King's turn:
Penguin King is now targeting Coach
Penguin King's attack hits Coach.
* Coach took 2 total damage:
   - 2 Slashing damage
* Coach now has 8 HP left.
Penguin King gains advantage. Advantage count is now 1.
Penguin King's attack hits Coach.
* Coach took 2 total damage:
   - 2 Slashing damage
* Coach now has 6 HP left.
Penguin King's attack hits Coach.
* Coach took 2 total damage:
   - 2 Slashing damage
* Coach now has 4 HP left.

Coach's turn:
Coach is now targeting Penguin King
Coach's attack hits Penguin King.
* Penguin King took 3 total damage:
   - 3 Slashing damage
* Penguin King now has 7 HP left.
Coach gains advantage. Advantage count is now 1.
Coach's attack hits Penguin King.
* Penguin King took 2 total damage:
   - 2 Slashing dama

defender :  Coach: physical_defense: 8, mystical_defense: 8, max_health_points: 10, move_speed: 5, max_action_points: 4, max_attack_before_penalty: 1, critical_hit_damage: 2, critical_hit_threshold: 20, heavy_hit_damage: 1, brutal_hit_damage: 1, mastery_light_armor: True
defender :  Coach: physical_defense: 8, mystical_defense: 8, max_health_points: 10, move_speed: 5, max_action_points: 4, max_attack_before_penalty: 1, critical_hit_damage: 2, critical_hit_threshold: 20, heavy_hit_damage: 1, brutal_hit_damage: 1, mastery_light_armor: True
defender :  Coach: physical_defense: 8, mystical_defense: 8, max_health_points: 10, move_speed: 5, max_action_points: 4, max_attack_before_penalty: 1, critical_hit_damage: 2, critical_hit_threshold: 20, heavy_hit_damage: 1, brutal_hit_damage: 1, mastery_light_armor: True
defender :  Penguin King: physical_defense: 8, mystical_defense: 8, max_health_points: 10, move_speed: 5, max_action_points: 4, max_attack_before_penalty: 1, critical_hit_damage: 2, cr

In [5]:
# Creating some damage modifiers
fire_vulnerability = rpg.Trait(name="fire_vulnerability",
                               damage_modifiers=rpg.Vulnerability(damage_type=rpg.Fire(), 
                                                                  value=2, is_multiplicative=True))
fire_resistance = rpg.Trait(name="fire_resistance",
                            damage_modifiers=rpg.Resistance(damage_type=rpg.Fire(), value=1))
fire_immune = rpg.Trait(name="fire_immune",
                        damage_modifiers=rpg.Resistance(damage_type=rpg.Fire(), 
                                                        value=0, is_multiplicative=True))

In [6]:
# Creating an epic sword

fire_damage = rpg.Damage(damage_type=rpg.Fire(), value=4)
physical_damage = rpg.Damage(damage_type=rpg.Slashing(), value=2)

# Multiple ways to create a sword
sword = rpg.Weapon(name="Flaming Sword", weapon_range=1, damages=[physical_damage, fire_damage])
sword.weapon_styles = [rpg.Sword_style()]

sword = rpg.MeleeWeapon(name="Flaming Sword", damages=[physical_damage, fire_damage])
sword.weapon_styles = [rpg.Sword_style()]

sword = rpg.MeleeWeapon(name="Flaming Sword", weapon_styles=rpg.Sword_style(), 
                        damages=[physical_damage, fire_damage])

sword = rpg.Sword(name="Flaming Sword", damages=[physical_damage, fire_damage])

# Adding damage modifiers to items
sword.add_trait([fire_vulnerability])

print([trait.name for trait in sword.traits])


['fire_vulnerability']


In [7]:
# Creating some items
ring_trait = rpg.Trait(name="Epic Ring Trait",
                        damage_modifiers=rpg.Resistance(damage_type=rpg.Fire(), 
                                                        value=0, is_multiplicative=True),
                        initiative=2, critical_hit_threshold=-1)

epic_ring = rpg.Item(name="Epic Ring", traits=ring_trait)
epic_ring = rpg.Item(name="Epic Ring", traits=[ring_trait])
epic_ring = rpg.Item(name="Epic Ring")
epic_ring.add_trait([ring_trait])

shield = rpg.Shield(name="Targe", physical_defense=2, physical_damage_reduction=1)

spellDC_trait = rpg.Trait(spell_DC=1, name= "Spell DC increase")  # Increase your spell DC by 1
magic_robes = rpg.Armor(name="Magic Robes of spell DC", physical_defense=1, traits=[spellDC_trait])


heavy_armor = rpg.Armor(physical_defense=3, traits=[fire_resistance], name="Plate Armor")

print("epic_ring", [trait.name for trait in epic_ring.traits])
print("magic_robes", [trait.name for trait in magic_robes.traits])
print("heavy_armor", [trait.name for trait in heavy_armor.traits])

epic_ring ['Epic Ring Trait']
magic_robes ['Spell DC increase']
heavy_armor ['fire_resistance']


In [8]:
print(shield.attributes)
print(magic_robes.attributes)
print(heavy_armor.attributes)

Attributes(physical_defense=2, mystical_defense=0, physical_damage_reduction=1, mystical_damage_reduction=0, max_health_points=0, max_stamina_points=0, max_grit_points=0, max_mana_points=0, might=0, agility=0, intelligence=0, charisma=0, prime_modifier_bonus=0, might_save=0, agility_save=0, intelligence_save=0, charisma_save=0, might_check=0, agility_check=0, intelligence_check=0, charisma_check=0, initiative=0, move_speed=0, might_adv=0, agility_adv=0, intelligence_adv=0, charisma_adv=0, combat_mastery=0, spell_DC=0, max_action_points=0, max_attack_before_penalty=0, critical_hit_damage=0, critical_hit_threshold=0, heavy_hit_damage=0, brutal_hit_damage=0, death_threshold=0, mastery_light_armor=False, mastery_heavy_armor=False, is_magic=False, is_bleeding=False, is_blinded=False, is_burning=False, is_charmed=False, is_dazed=False, is_deafened=False, is_doomed=False, is_exposed=False, is_frightened=False, is_grappled=False, is_hindered=False, is_impaired=False, is_incapacitated=False, is

In [9]:
# Creating an actor  (stats are 0 by default)
hero_1 = rpg.Actor(max_health_points=20, 
                   combat_mastery=3, might=3, max_mana_points=5, name="hero_1")
hero_2 = rpg.Actor(max_health_points=20, combat_mastery=3, might=3, name="hero_2")

# Some status effects : the name is required for now to differentiate when a similar effect is added twice
bleeding = rpg.Trait(duration=4, is_bleeding=True, name="Small bleeding")
blessed = rpg.Trait(duration=4, is_magic=True, name="Benediction")

sword.add_trait(blessed)



# Attributes are dynamically added bia setters to the Actor class :
print(hero_1.traits)
print(hero_1.attributes.might)
print(hero_1.might)
print(hero_1.current_mana_points)
print(hero_1.max_mana_points)

print(hero_1.current_action_points)
print(hero_1.current_health_points)
print(hero_1.combat_mastery)

print("hero_1 traits", [trait.name for trait in hero_1.traits])

hero_1.add_item(sword)

print("sword.is_magic ?", sword.is_magic)
print("hero_1.is_magic ?", hero_1.is_magic)
print("hero_1.has_magic_weapon ?", hero_1.has_magic_weapon)
print("hero_1.is_magic ?", hero_1.base_attributes.is_magic)

hero_2.add_item(sword)

[]
3
3
5
5
4
20
3
hero_1 traits []
sword.is_magic ? True
hero_1.is_magic ? True
hero_1.has_magic_weapon ? True
hero_1.is_magic ? False


In [ ]:
hero_great = rpg.Actor(max_health_points=20, 
                   combat_mastery=3, might=3, max_mana_points=5, name="hero_great")
print("PD : ", hero_great.physical_defense, 
      "MD : ", hero_great.physical_damage_reduction)
hero_great.add_item([shield, magic_robes, sword])  # items are unique, and cannot be added twice to an actor
print("PD : ", hero_great.physical_defense, 
      "MD : ", hero_great.physical_damage_reduction)

print("hero_1 traits", [trait.name for trait in hero_great.traits])

print("shield", shield.damage_modifiers)
print("magic_robes", magic_robes.damage_modifiers)
print("sword", sword.damage_modifiers)

print("hero_1 traits", [trait.name for trait in hero_great.traits])
print("hero_great.traits", hero_great.traits)
print(len(hero_great.get_trait_sources()))
print("hero_1 traits", [trait.name for trait in hero_great.traits])
print(len(hero_great.get_trait_sources()))
print("hero_1 traits", [trait.name for trait in hero_great.traits])

In [ ]:
print(hero_2.traits)
hero_2.add_trait(bleeding)
print(hero_2.traits)
hero_2.new_round()
print(hero_2.traits)
print(hero_2.traits)

In [ ]:
print("Attributes:", hero_great.attributes)
print("Items:", hero_great.item_manager.get_items())
print("Items:", hero_great.item_manager.get_items())
print("Items:", hero_great.item_manager.get_items())
print("Vulnerabilities:", hero_great.vulnerabilities)

print("Items:", [item.traits for item in hero_great.item_manager.get_items()])

In [ ]:
print("Weapons:", sword.vulnerabilities)
print("Traits:", [trait.name for trait in hero_great.traits])
print("Vulnerabilities:", hero_great.vulnerabilities)
print("Vulnerabilities:", hero_great.vulnerabilities)
print("Vulnerabilities:", hero_great.vulnerabilities)

In [ ]:
# Display resistances and vulnerabilities

print("Sword", sword.attributes)
print("Weapons:", hero_great.weapons)
print("Resistances:", hero_great.resistances)
print("Vulnerabilities:", hero_great.vulnerabilities)
print("Modifiers:", hero_great.damage_modifiers)
print("Vulnerabilities:", hero_great.vulnerabilities)
print("Vulnerabilities:", hero_great.vulnerabilities)
print("Weapons:", hero_great.weapons)


In [ ]:
# Simulate taking damage
damages = [fire_damage, physical_damage]
hero_great.take_damage(damages)

print("physical_defense:", hero_great.physical_defense)
print("mystical_defense:", hero_great.mystical_defense)
print("health_points:", hero_great.current_health_points)
print("modifiers:", hero_great.damage_modifiers)
print("mana:", hero_great.current_mana_points)

In [ ]:
#   Try attacks
hero_1.full_rest()
hero_2.full_rest()
print("mana:", hero_1.current_mana_points, hero_1.max_mana_points)
attack = rpg.Attack()
attack.execute(hero_1, [hero_2])  # we can wrap this nicely
print("mana:", hero_1.current_mana_points)
print(hero_1.advantage_count)
gain_advantage = rpg.GainAdvantage()
gain_advantage.execute(hero_1, [hero_1])
print(hero_1.advantage_count)

In [ ]:
# Create a Fireball spell

damage_action_success = rpg.InflictDamage(
    damages=[rpg.Damage(damage_type=rpg.Fire(), value=10)])

damage_action_failure = rpg.InflictDamage(
    damages=[rpg.Damage(damage_type=rpg.Fire(), value=20)])

fire_condition = rpg.Trait(duration=5, is_burning=True, name="Burning")

fireball_action = rpg.ImposeSavingThrow(
    stat='Agility',
    difficulty=15,
    on_success=[damage_action_success],
    on_failure=[damage_action_failure, fire_condition]
)

fireball = rpg.Spell(actions=[fireball_action], school="Elemental", name="Fireball")
fireball.execute(hero_1, [hero_2])


In [ ]:
fireball.execute(hero_1, [hero_2])

print("modifiers:", hero_2.modifiers)


## Test combat initiative

In [ ]:
# Example usage:
players = [rpg.Actor(name="Player 1", health_points=21, Might=4, equipped_items=[sword]),
           rpg.Actor(name="Player 2", health_points=22, Might=4, equipped_items=[sword])]
monsters = [rpg.Actor(name="Monster 1", health_points=21, Might=4, equipped_items=[sword]),
            rpg.Actor(name="Monster 2", health_points=22, Might=4, equipped_items=[sword])]
combat_manager = rpg.CombatManager(players, monsters, dc=1)

combat_manager.roll_initiative()


# Test combat

In [ ]:
combat_manager.reset_combat()
num_rounds, num_turns, winning_team, remaining_hp = combat_manager.run_combat()
print("num_rounds = ", num_rounds)
print("num_turns = ", num_turns)
print("winning_team = ", winning_team)
print(combat_manager.fight_debrief())


# Statistical results

In [ ]:
players = [rpg.Actor(name="Hero 1", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                      target_mode="target_weakest", strategy=rpg.HelpAllyStrategy()),
           rpg.Actor(name="Hero 2", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                      target_mode="target_weakest", strategy=rpg.HelpAllyStrategy())]

monsters = [rpg.Actor(name="Wolf 1", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                       target_mode="target_weakest", strategy=rpg.DefaultStrategy()),
            rpg.Actor(name="Wolf 2", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                       target_mode="target_weakest", strategy=rpg.DefaultStrategy())]

num_rounds_list, num_turns_list, winrate_A, remaining_hp_list = rpg.run_simulations(5000, players, monsters, dc=15)
print(winrate_A)

In [ ]:
rpg.plot_simulation_results(num_turns_list, winrate_A, remaining_hp_list)